# FILTER

In [ ]:
import pandas as pd
from Bio import SeqIO


In [ ]:
input_df_counts_paired=snakemake.input.df_counts_paired
input_merged_summary=snakemake.input.merged_summary
input_vOTUs_prefiltered=snakemake.input.vOTUs_prefiltered
input_vibrant_circular=snakemake.input.vibrant_circular
input_vibrant_positive=snakemake.input.vibrant_positive
input_vibrant_quality=snakemake.input.vibrant_quality
input_vibrant_summary=snakemake.input.vibrant_summary
input_virsorter_table=snakemake.input.virsorter_table
input_virsorter_positive_list=snakemake.input.virsorter_positive_list
input_genomad_virus_summary=snakemake.input.genomad_virus_summary
input_genomad_plasmid_summary=snakemake.input.genomad_plasmid_summary
input_genomad_viral_fasta=snakemake.input.genomad_viral_fasta
input_genomad_viral_fasta_conservative=snakemake.input.genomad_viral_fasta_conservative
#________
output_summary=snakemake.output.summary

SAMPLES=snakemake.params.samples

In [ ]:
checkv_df= pd.read_csv(input_merged_summary, sep="\t",
                            names= ["contig_id_vOTU", "contig_length_vOTU", "provirus_vOTU", "proviral_length_vOTU",
                           "gene_count_vOTU", "viral_genes_vOTU", "host_genes_vOTU", "checkv_quality_vOTU",
                           "miuvig_quality_vOTU", "completeness_vOTU", "completeness_method_vOTU",
                           "contamination_vOTU", "kmer_freq_vOTU", "warnings_vOTU"])
checkv_df

In [ ]:
vOTU_list=[]
for record in SeqIO.parse(input_vOTUs_prefiltered, "fasta"): vOTU_list.append(record.id)
vOTU_prefiltered=pd.DataFrame(vOTU_list, columns=["vOTU_name"])
checkv_df= pd.read_csv(input_merged_summary, sep="\t",
                            names= ["checkV_contig_id", "checkV_contig_length", "checkV_provirus", "checkV_proviral_length",
                           "checkV_gene_count", "checkV_viral_genes", "checkV_host_genes", "checkv_quality",
                           "checkV_miuvig_quality", "checkV_completeness", "checkV_completeness_method",
                           "checkV_contamination", "checkV_kmer_freq", "checkV_warnings"])
vibrant_circular_df=pd.read_csv(input_vibrant_circular, names=["VIBRANT_circular_name"]); vibrant_circular_df["VIBRANT_circular"]="Y"
vibrant_positive_df=pd.read_csv(input_vibrant_positive, names=["VIBRANT_positive_name"]); vibrant_positive_df["VIBRANT_positive"]="Y"
vibrant_quality_df=pd.read_csv(input_vibrant_quality, sep ="\t"); vibrant_quality_df.columns=["VIBRANT_scaffold","VIBRANT_lifecycle","VIBRANT_quality",]; vibrant_quality_df=vibrant_quality_df.groupby("VIBRANT_scaffold").first()
vibrant_summary_df=pd.read_csv(input_vibrant_summary, sep ="\t") ; vibrant_summary_df.columns = ['VIBRANT_' + str(col).replace(" " , "_") for col in vibrant_summary_df.columns]
virsorter_summary_df=pd.read_csv(input_virsorter_table, sep ="\t"); virsorter_summary_df.columns = ['VirSorter2_' + str(col).replace(" " , "_") for col in virsorter_summary_df.columns]; virsorter_summary_df["VirSorter2_positive"]="Y"
genomad_summary_df=pd.read_csv(input_genomad_virus_summary, sep ="\t"); genomad_summary_df.columns = ['geNomad_' + str(col).replace(" " , "_") for col in genomad_summary_df.columns]

viral_list=[]
for record in SeqIO.parse(input_genomad_viral_fasta, "fasta"): viral_list.append(record.id)
geNomad_viral=pd.DataFrame(viral_list, columns=["geNomad_viral_name"]); geNomad_viral["geNomad_viral"]="Y"
viral_list_conservative=[]
for record in SeqIO.parse(input_genomad_viral_fasta_conservative, "fasta"): viral_list_conservative.append(record.id)
geNomad_viral_conservative=pd.DataFrame(viral_list_conservative, columns=["geNomad_viral_conservative_name"])
geNomad_viral_conservative["geNomad_viral_conservative"]="Y"


In [ ]:
df_temp1=vOTU_prefiltered.merge(vibrant_circular_df, left_on="vOTU_name", right_on="VIBRANT_circular_name", how="left")
df_temp2=df_temp1.merge(vibrant_positive_df, left_on="vOTU_name", right_on="VIBRANT_positive_name", how="left")
df_temp3=df_temp2.merge(vibrant_quality_df, left_on="vOTU_name", right_on="VIBRANT_scaffold", how="left")
df_temp4=df_temp3.merge(vibrant_summary_df, left_on="vOTU_name", right_on="VIBRANT_scaffold", how="left")
df_temp5=df_temp4.merge(virsorter_summary_df, left_on="vOTU_name", right_on="VirSorter2_seqname", how="left")
df_temp6=df_temp5.merge(genomad_summary_df, left_on="vOTU_name", right_on="geNomad_seq_name", how="left")
df_temp7=df_temp6.merge(geNomad_viral, left_on="vOTU_name", right_on="geNomad_viral_name", how="left")
df_temp8=df_temp7.merge(geNomad_viral_conservative, left_on="vOTU_name", right_on="geNomad_viral_conservative_name", how="left")
df_merged=df_temp8.merge(checkv_df, left_on="vOTU_name", right_on="checkV_contig_id", how="left")
df_merged=df_merged.drop(["VIBRANT_circular_name", "VIBRANT_positive_name", "VIBRANT_scaffold", "checkV_contig_id"], axis=1)
df_merged

# How many genomad, genomad conservative, virsorter, viridic
#organize so they are at the beggining with the length

In [ ]:
df_merged.to_csv(output_summary)

In [ ]:
df=df_merged[["VirSorter2_positive","VIBRANT_positive","geNomad_viral","geNomad_viral_conservative"]]

              
import matplotlib.pyplot as plt
from matplotlib_venn import venn3

# Assuming your DataFrame is named df
# Get the counts for each category
a_count = df['VirSorter2_positive'].value_counts()['Y']
b_count = df['VIBRANT_positive'].value_counts()['Y']
c_count = df['geNomad_viral_conservative'].value_counts()['Y']
ab_count = df[df['VirSorter2_positive'] == 'Y']['VIBRANT_positive'].value_counts()['Y']
ac_count = df[df['VirSorter2_positive'] == 'Y']['geNomad_viral_conservative'].value_counts()['Y']
bc_count = df[(df['VIBRANT_positive'] == 'Y') & (df['geNomad_viral_conservative'] == 'Y')]['VIBRANT_positive'].value_counts()['Y']
abc_count = df[(df['VirSorter2_positive'] == 'Y') & (df['VIBRANT_positive'] == 'Y') & (df['geNomad_viral_conservative'] == 'Y')]['VIBRANT_positive'].value_counts()['Y']

# Create the Venn diagram
venn_labels = {'100': a_count - ab_count - ac_count + abc_count,
               '010': b_count - ab_count - bc_count + abc_count,
               '001': c_count - ac_count - bc_count + abc_count,
               '110': ab_count - abc_count,
               '101': ac_count - abc_count,
               '011': bc_count - abc_count,
               '111': abc_count}

plt.figure(figsize=(8, 6))
venn_diagram = venn3(subsets=venn_labels, set_labels=('VirSorter2_positive', 'VIBRANT_positive', 'geNomad_viral_conservative'))

# Add the counts to the Venn diagram
for text in venn_diagram.set_labels:
    text.set_fontsize(14)
for text in venn_diagram.subset_labels:
    if text:
        text.set_fontsize(12)

# Show the Venn diagram
plt.title("Venn Diagram of Categories 'VirSorter2_positive', 'VIBRANT_positive', and 'geNomad_viral_conservative'")
plt.show()


In [ ]:
df=df_merged[["VirSorter2_positive","VIBRANT_positive","geNomad_viral","geNomad_viral_conservative"]]

              
import matplotlib.pyplot as plt
from matplotlib_venn import venn3

# Assuming your DataFrame is named df
# Get the counts for each category
a_count = df['VirSorter2_positive'].value_counts()['Y']
b_count = df['VIBRANT_positive'].value_counts()['Y']
c_count = df['geNomad_viral'].value_counts()['Y']
ab_count = df[df['VirSorter2_positive'] == 'Y']['VIBRANT_positive'].value_counts()['Y']
ac_count = df[df['VirSorter2_positive'] == 'Y']['geNomad_viral'].value_counts()['Y']
bc_count = df[(df['VIBRANT_positive'] == 'Y') & (df['geNomad_viral'] == 'Y')]['VIBRANT_positive'].value_counts()['Y']
abc_count = df[(df['VirSorter2_positive'] == 'Y') & (df['VIBRANT_positive'] == 'Y') & (df['geNomad_viral'] == 'Y')]['VIBRANT_positive'].value_counts()['Y']

# Create the Venn diagram
venn_labels = {'100': a_count - ab_count - ac_count + abc_count,
               '010': b_count - ab_count - bc_count + abc_count,
               '001': c_count - ac_count - bc_count + abc_count,
               '110': ab_count - abc_count,
               '101': ac_count - abc_count,
               '011': bc_count - abc_count,
               '111': abc_count}

plt.figure(figsize=(8, 6))
venn_diagram = venn3(subsets=venn_labels, set_labels=('VirSorter2_positive', 'VIBRANT_positive', 'geNomad_viral'))

# Add the counts to the Venn diagram
for text in venn_diagram.set_labels:
    text.set_fontsize(14)
for text in venn_diagram.subset_labels:
    if text:
        text.set_fontsize(12)

# Show the Venn diagram
plt.title("Venn Diagram of Categories 'VirSorter2_positive', 'VIBRANT_positive', and 'geNomad_viral'")
plt.show()


In [ ]:
df_counts_paired=pd.read_csv(input_df_counts_paired, index_col=0)
df_counts_paired

In [ ]:
# reads_df=pd.read_html(work_dir +"FIGURES_AND_TABLES/01_post_qc_read_summary.tot.html")[0].set_index('Unnamed: 0').reset_index().drop(columns=['Unnamed: 0'])[["sample","number_post"]].groupby("sample").first()
# reads_df=df_counts_paired[["sample", "bbduk"]]

# mapped_pair_f=[]
# mapped_pair_u=[]
# mapped_pair_v=[]
# mapped_pair_a=[]
# mapped_pair_4=[]

# for sample in SAMPLES:
#     file_f = open(mapping_dir + "/bbmap_flagstats_" + sample + "." + sampling + '.txt')  
#     file_u = open(mapping_dir + "/bbmap_flagstats_" + sample + "_stat_unfiltered." + sampling + '.txt')  
#     file_v = open(mapping_dir + "/bbmap_flagstats_" + sample + "_stat_viral_contigs." + sampling + '.txt')  
#     file_a = open(mapping_dir + "/bbmap_flagstats_" + sample + "_stat_assembled_contigs." + sampling + '.txt')  
# #     file_4 = open(work_dir+ '06_MAPPING/bbmap_flagstats_' + sample + "." + sampling + '.txt')  


# # read the content of the file opened
#     content_f = file_f.readlines()
#     content_u = file_u.readlines()
#     content_v = file_v.readlines()
#     content_a = file_a.readlines()
# #     content_4 = file_4.readlines()

#     mapped_pair_f.append(int(content_f[9].split()[0]))
#     mapped_pair_u.append(int(content_u[9].split()[0]))
#     mapped_pair_v.append(int(content_v[9].split()[0]))
#     mapped_pair_a.append(int(content_a[9].split()[0]))
# #     mapped_pair_4.append(int(content_4[9].split()[0]))

# df_mapped=pd.DataFrame()
# df_mapped["sample"]=SAMPLES
# df_mapped["mapped_f"]=mapped_pair_f
# df_mapped["mapped_u"]=mapped_pair_u
# df_mapped["mapped_v"]=mapped_pair_v
# df_mapped["mapped_a"]=mapped_pair_a
# # df_mapped["mapped_4"]=mapped_pair_4

# df_mapped=df_mapped.merge(reads_df,left_on="sample", right_on="sample")

# df_mapped["%mapped assembled"]=df_mapped["mapped_a"]*100/df_mapped["number_post"]
# df_mapped["%mapped viral"]=df_mapped["mapped_v"]*100/df_mapped["number_post"]
# df_mapped["%mapped unfiltered vOTUs"]=df_mapped["mapped_u"]*100/df_mapped["number_post"]
# # df_mapped["%mapped filtered 4kb"]=df_mapped["mapped_4"]*100/df_mapped["number_post"]
# df_mapped["%mapped filtered"]=df_mapped["mapped_f"]*100/df_mapped["number_post"]

# df_mapped["diff viral filtered"]=df_mapped["%mapped filtered"]-df_mapped["%mapped viral"]
# # df_mapped["diff unfiltered filtered_4"]=df_mapped["%mapped unfiltered vOTUs"]-df_mapped["%mapped filtered 4kb"]
# # df_mapped["diff_10_4"]=df_mapped["%mapped filtered 4kb"]-df_mapped["%mapped filtered 10kb"]
# df_mapped.loc['mean'] = df_mapped.mean()

# df_mapped=df_mapped.drop(["mapped_f","mapped_u","mapped_v","mapped_a"], axis=1)
# df_mapped=df_mapped.round(1)
# df_mapped
# df_mapped.style.background_gradient(cmap="RdYlGn").format({'Quantity': "{:.2%}"})
# # df_mapped.style.background_gradient(cmap="RdYlGn", vmin=0)